## Import libraries

In [254]:
import os 
import csv
import numpy as np
import pandas as pd
import random # library to generate random numbers
np.random.seed(seed=42)
import math
from datetime import datetime

In [255]:
#!dir

 Volume in drive E has no label.
 Volume Serial Number is 9E09-BCBF

 Directory of E:\Batch72_Rennes_BLR\CSE7321c\Lab\20190908_Batch66_CSE7321c_Lab01_GA

11/12/2019  11:29 PM    <DIR>          .
11/12/2019  11:29 PM    <DIR>          ..
11/12/2019  05:50 PM    <DIR>          .ipynb_checkpoints
09/08/2019  02:20 PM               705 cities_and_distances.csv
11/12/2019  05:53 PM               779 europen_Cities.csv
09/08/2019  02:20 PM         6,751,860 Genetic Algorithm - TSP Problem.pptx
11/12/2019  11:29 PM            59,423 Genetic_Algorithm_TSP_Solution-EuropenCities.ipynb
11/12/2019  05:43 PM            42,373 Genetic_Algorithm_TSP_Solution.ipynb
11/12/2019  11:30 PM            22,466 map13.html
               6 File(s)      6,877,606 bytes
               3 Dir(s)  503,606,091,776 bytes free


#### Import data

In [256]:
data1 = pd.read_csv("europen_Cities.csv",header=0,index_col=0)
data1

,Athens,Barcelona,Brussels,Calais,Cherbourg,Cologne,Copenhagen,Geneva,Gibraltar,Hamburg,Hook of Holland
Athens,0,3313,2963,3175,3339,2762,3276,2610,4485,2977,3030
Barcelona,3313,0,1318,1326,1294,1498,2218,803,1172,2018,1490
Brussels,2963,1318,0,204,583,206,966,677,2256,597,172
Calais,3175,1326,204,0,460,409,1136,747,2224,714,330
Cherbourg,3339,1294,583,460,0,785,1545,853,2047,1115,731
Cologne,2762,1498,206,409,785,0,760,1662,2436,460,269
Copenhagen,3276,2218,966,1136,1545,760,0,1418,3196,460,269
Geneva,2610,803,677,747,853,1662,1418,0,1975,1118,895
Gibraltar,4485,1172,2256,2224,2047,2436,3196,1975,0,2897,2428
Hamburg,2977,2018,597,714,1115,460,460,1118,2897,0,550


#### Starting Point is Bangalore

In [257]:
starting_point = "Athens"

###### The number of possible solutions are: 10! = 3628800
We can generate all possible solutions and try to find the best of those , but this is a time taking process and may not be feasible for big problems.
Genetic algorithm approach takes a randominzed search approach to find the best solution, with enough number of iterations, mutation and cross over, GA can reach the global minima for most of the problems.

##### Map places to integers

In [258]:
cities_mapping = {}

for i in enumerate(data1.columns):
    cities_mapping[i[0]] = i[1]
cities_mapping

{0: 'Athens',
 1: 'Barcelona',
 2: 'Brussels',
 3: 'Calais',
 4: 'Cherbourg',
 5: 'Cologne',
 6: 'Copenhagen',
 7: 'Geneva',
 8: 'Gibraltar',
 9: 'Hamburg',
 10: 'Hook of Holland'}

In [259]:
one_route = [5,6,10,9,1,3,4,7,2,8]# one of the routes

In [260]:
one_route

[5, 6, 10, 9, 1, 3, 4, 7, 2, 8]

## createRandomRoute:: this function to generate random route

In [308]:
def createRandomRoute():
    """ This function generates a route by sampling numbers """
    my_randoms = random.sample(range(1,11), 10)    # excluding 0 from the range as 0 is the starting point for us i.e Bangalore
    return my_randoms

In [309]:
temp_route = createRandomRoute()

In [310]:
temp_route

[2, 6, 7, 10, 9, 3, 5, 4, 8, 1]

## fitnessFunction :: Function to compute the route cost using the already defined path 

In [311]:
def fitnessFunction(pvRouteMap):
    """This functions takes the route generated and returns the cost incurred."""
    traverseData    = data1.copy() # creating a copy of the original df
    sourcePoint     = 0 # defining a starting point
    stopsCovered    = 0 # setting the number of stops covered to 0
    routeCost       = 0 # setting the initial cost to 0
    pvRouteMap1 = pvRouteMap #+ [pvRouteMap[0]]
#     print(traverseData)
    # initiate a while loop and calculate the cost for the whole path traversed and return the cost
    while(stopsCovered < len(pvRouteMap1)) :  
        routeCost = routeCost + traverseData.iloc[sourcePoint, pvRouteMap1[stopsCovered]]   
        """route cost is the sum of the cost incurred from travelling from one point to the next according
        to the route that was generated previously."""
        sourcePoint = pvRouteMap1[stopsCovered] # update the source point to the next point in the route
        stopsCovered = stopsCovered+1 #Adding 1 to the stops covered
#         print(sourcePoint)
    routeCost = routeCost + traverseData.iloc[pvRouteMap1[-1],0]

    return(routeCost)

In [312]:
print(temp_route)

fitnessFunction(temp_route)


[2, 6, 7, 10, 9, 3, 5, 4, 8, 1]


15232

#### We can look at the positions and think of one good route , we will see if our algorithm performs better than this route

In [313]:
one_route_cities = []
for i in one_route:
    one_route_cities.append(cities_mapping[i])
one_route_cities

['Cologne',
 'Copenhagen',
 'Hook of Holland',
 'Hamburg',
 'Barcelona',
 'Calais',
 'Cherbourg',
 'Geneva',
 'Brussels',
 'Gibraltar']

##  mutateRoute:: this function will generate mutated path where we define mutation policy.

In [314]:
def mutateRoute(initPath, mutateFactor):
    """ This functions generates a mutated path , takes an input path and returns a mutated path.
    Mutate factor is the point at which the string (route here) is split and the two parts are swapped.
    
        Mutation
        Step 1: Pick a parent chromosome
        Step 2: Randomly pick an index (position in the parent chromosome)
        Step 3: Swap the values before and after the index to generate the mutated chromosome (child).
        
    Ex: 
        input to the function:        
            parent = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10]
            index = 3 (randomy generate the index)
            
        with in function
            temp1 = [4, 6, 2]
            temp2 = [8, 3, 5, 7, 1, 9, 10]
        
        output:
            child  = [8, 3, 5, 7, 1, 9, 10, 4, 6, 2]

    Implemetation:
        getMutatedPath(initPath = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10], mutateFactor = 3)
        output: [8, 3, 5, 7, 1, 9, 10, 4, 6, 2].
    """
    
    temp1 = initPath[0:mutateFactor]
    temp2 = initPath[mutateFactor:len(initPath)]
    newPath = temp2 + temp1

    return(newPath)

In [315]:
print(mutateRoute.__doc__)

 This functions generates a mutated path , takes an input path and returns a mutated path.
    Mutate factor is the point at which the string (route here) is split and the two parts are swapped.
    
        Mutation
        Step 1: Pick a parent chromosome
        Step 2: Randomly pick an index (position in the parent chromosome)
        Step 3: Swap the values before and after the index to generate the mutated chromosome (child).
        
    Ex: 
        input to the function:        
            parent = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10]
            index = 3 (randomy generate the index)
            
        with in function
            temp1 = [4, 6, 2]
            temp2 = [8, 3, 5, 7, 1, 9, 10]
        
        output:
            child  = [8, 3, 5, 7, 1, 9, 10, 4, 6, 2]

    Implemetation:
        getMutatedPath(initPath = [4, 6, 2, 8, 3, 5, 7, 1, 9, 10], mutateFactor = 3)
        output: [8, 3, 5, 7, 1, 9, 10, 4, 6, 2].
    


#### Generate a sample path and mutate that path 

In [316]:
temp_route = createRandomRoute()

In [317]:
temp_route

[8, 6, 3, 5, 2, 9, 7, 10, 4, 1]

In [318]:
mutateRoute(temp_route,2)

[3, 5, 2, 9, 7, 10, 4, 1, 8, 6]

In [319]:
fitnessFunction(temp_route)

17380

In [320]:
print("Total distance travelled to cover the route of {} is {}".format(" => ".join( one_route_cities ),fitnessFunction(one_route)))

Total distance travelled to cover the route of Cologne => Copenhagen => Hook of Holland => Hamburg => Barcelona => Calais => Cherbourg => Geneva => Brussels => Gibraltar is 16416


# Function to create Initial (random) Population 

In [321]:
def initialPopCost(ninitpop = 10):
    """This function generates new routes and calculates the cost of the route.
    returns a dictionary of routes and the cost."""
    np.random.seed(seed=42)
    intial_cost = 0
    routeCost = []
    routes = [createRandomRoute() for i in range(0, ninitpop)]
    routes_Cost = [fitnessFunction(i) for i in routes]
    routes_DF = pd.DataFrame([routes,routes_Cost]).transpose()
    routes_DF.columns = ['Route','Cost']
    return(routes_DF)
# initialPopCost()

In [322]:
initial_pop_cost = initialPopCost(10)
sorted_init_pop = initial_pop_cost.sort_values(['Cost'])

In [323]:
initial_pop_cost

,Route,Cost
0,"[6, 7, 2, 9, 4, 3, 1, 8, 10, 5]",15500
1,"[2, 5, 4, 3, 1, 9, 7, 10, 6, 8]",17721
2,"[1, 4, 9, 5, 8, 3, 6, 2, 10, 7]",16621
3,"[4, 1, 3, 6, 8, 9, 7, 5, 10, 2]",19372
4,"[6, 5, 7, 3, 2, 9, 10, 1, 4, 8]",17112
5,"[5, 2, 10, 6, 3, 9, 1, 8, 4, 7]",13959
6,"[6, 4, 7, 3, 9, 1, 5, 2, 8, 10]",18571
7,"[9, 1, 3, 7, 5, 4, 6, 8, 10, 2]",19819
8,"[5, 2, 1, 8, 9, 3, 10, 7, 6, 4]",16596
9,"[2, 3, 8, 7, 6, 1, 4, 5, 10, 9]",16877


In [324]:
sorted_init_pop.head()

,Route,Cost
5,"[5, 2, 10, 6, 3, 9, 1, 8, 4, 7]",13959
0,"[6, 7, 2, 9, 4, 3, 1, 8, 10, 5]",15500
8,"[5, 2, 1, 8, 9, 3, 10, 7, 6, 4]",16596
2,"[1, 4, 9, 5, 8, 3, 6, 2, 10, 7]",16621
9,"[2, 3, 8, 7, 6, 1, 4, 5, 10, 9]",16877


## Partially Mapped Crossover

###### - implemented PMX conceptualized by goldberg

In [388]:
def crossOverFunction(parent1, parent2,crossover_factor_start_pos=2,
                             crossover_factor_end_pos=6):
    
    """This function implements the partially mapped crossover by goldeberg (https://www.hindawi.com/journals/cin/2017/7430125/)
    
    inputs: 2 parent solutions, crossover start position, crossover end position.
    crossover start position and  crossover end position are randomly generated in the solution.
    
    output: 2 child solutions, i.e the crossedover solutions.
    Example :
    
        Inputs:
            p1 = [5,7,2,4,8,9,3,6]
            p2 = [8,6,9,3,2,5,7,4]
            crossover_start_pos 2 (randomly selected)
            crossover_end_pos 6 (randomly selected)
        Internals:
            The mapping created is [(2, 9), (4, 3), (8, 2), (9, 5)]
            Child1 Intermediate [9, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [2, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [8, 7, 9, 3, 2, 5, 4, 6]
            Child1 final [8, 7, 9, 3, 2, 5, 4, 6]
            Child2 original [8, 6, 2, 4, 8, 9, 7, 4]
            Child2 Intermediate [2, 6, 2, 4, 8, 9, 7, 3]
            Child2 Intermediate [9, 6, 2, 4, 8, 9, 7, 3]
            Child2 Intermediate [5, 6, 2, 4, 8, 9, 7, 3]
            Child2 final [5, 6, 2, 4, 8, 9, 7, 3]
        
        Output:
            Child: [8, 7, 9, 3, 2, 5, 4, 6]
            Child: [5, 6, 2, 4, 8, 9, 7, 3].
    """
    
    indexes_for_crossover = random.sample(set(range(len(p1))), 2)
    print(indexes_for_crossover)
    crossover_factor_start_pos,crossover_factor_end_pos = min(indexes_for_crossover),max(indexes_for_crossover)
#     print (indexes_for_crossover)
    ## generate child 1
    child1 = parent1[0:crossover_factor_start_pos]+\
    parent2[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent1[crossover_factor_end_pos:]
    
    ## generate child 2
    child2 = parent2[0:crossover_factor_start_pos] +\
    parent1[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent2[crossover_factor_end_pos:]
    
    ## Create mappings
    mpping = list(zip(parent1[crossover_factor_start_pos:crossover_factor_end_pos], # create tuple of mappings
                      parent2[crossover_factor_start_pos:crossover_factor_end_pos]))
#     print(mpping)
    # run until all the nodes in the route are unique
    while len(np.unique(child1)) != len(child1):
        child1_part = child1[:crossover_factor_start_pos]+child1[crossover_factor_end_pos:]
        for i in child1_part:
            for j in mpping:
                if i == j[1]: #j[1] - Parent 2 window elements                 
                    child1_part[child1_part.index(i)] = j[0] #j[0] - Parent 1 window elements
                    
        child1 = child1_part[:crossover_factor_start_pos]+ child1[crossover_factor_start_pos:crossover_factor_end_pos]+ child1_part[crossover_factor_start_pos:]
        
#         print("Child1 Intermediate {}".format(child1))
#     print("Child1 final {}".format(child1))

#     print("Child2 original {}".format(child2))

    # run until all the nodes in the route are unique
    while len(np.unique(child2)) != len(child2):
        child2_part = child2[:crossover_factor_start_pos]+child2[crossover_factor_end_pos:]
        for i in child2_part:
            for j in mpping:
                if i == j[0]:
                    child2_part[child2_part.index(i)] = j[1]
        child2 = child2_part[:crossover_factor_start_pos]+child2[crossover_factor_start_pos:crossover_factor_end_pos]+child2_part[crossover_factor_start_pos:]
#         print("Child2 Intermediate {}".format(child2))
#     print("Child2 final {}".format(child2))
    return( child1,child2)

In [389]:
print(crossOverFunction.__doc__)

This function implements the partially mapped crossover by goldeberg (https://www.hindawi.com/journals/cin/2017/7430125/)
    
    inputs: 2 parent solutions, crossover start position, crossover end position.
    crossover start position and  crossover end position are randomly generated in the solution.
    
    output: 2 child solutions, i.e the crossedover solutions.
    Example :
    
        Inputs:
            p1 = [5,7,2,4,8,9,3,6]
            p2 = [8,6,9,3,2,5,7,4]
            crossover_start_pos 2 (randomly selected)
            crossover_end_pos 6 (randomly selected)
        Internals:
            The mapping created is [(2, 9), (4, 3), (8, 2), (9, 5)]
            Child1 Intermediate [9, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [2, 7, 9, 3, 2, 5, 4, 6]
            Child1 Intermediate [8, 7, 9, 3, 2, 5, 4, 6]
            Child1 final [8, 7, 9, 3, 2, 5, 4, 6]
            Child2 original [8, 6, 2, 4, 8, 9, 7, 4]
            Child2 Intermediate [2, 6, 2, 4, 8, 9, 7, 3

In [390]:
p1 = [5,7,2,4,8,9,3,6]

In [391]:
p2 = [8,6,9,3,2,5,7,4]

In [392]:
for i in zip([p1,p2],crossOverFunction(p1,p2)):
    print ("Parent: {}  : Child: {}".format(i[0],i[1]))

[1, 5]
Parent: [5, 7, 2, 4, 8, 9, 3, 6]  : Child: [5, 6, 9, 3, 2, 8, 4, 7]
Parent: [8, 6, 9, 3, 2, 5, 7, 4]  : Child: [9, 7, 2, 4, 8, 5, 6, 3]


 ### Best Solution So Far


In [330]:
def genetic_algorithm(sorted_current_generation, no_of_chromosomes = 100 ,mutation_factor=0.2, xover_factor = 0.3):
    
    nextgen_childs = [] 
    
    xover_pool = int(round(no_of_chromosomes * xover_factor ))
    no_of_crossovers = int((no_of_chromosomes // 2) )
    
    print ('Running crossover')
    for i in range(0,no_of_crossovers):
        parent_index1 = random.choice(range(0,xover_pool))
        p1= sorted_current_generation.iloc[parent_index1,0]
        parent_index2 = random.choice(range(0,xover_pool))
        p2= sorted_current_generation.iloc[parent_index2,0]

        c1, c2 = crossOverFunction(p1,p2)
        nextgen_childs.extend([c1,c2])
        
    nextgen_costs = [fitnessFunction(i) for i in nextgen_childs]
    next_generation = pd.DataFrame([nextgen_childs,nextgen_costs],).transpose() 
    next_generation.columns = ['Route','Cost']
    
    print('Running Mutation')
    for i in range(0, no_of_chromosomes) :
        mutation_prob = np.random.randn()
        if mutation_prob < mutation_factor:
            mutated_child = mutateRoute (next_generation.iloc[i,0] , 2)
            next_generation.iloc[i,0] = mutated_child
            next_generation.iloc[i,1] = fitnessFunction(next_generation.iloc[i,0])

    return (next_generation)

In [331]:
possible_solutions = math.factorial(10)

In [332]:
def get_final_rounte(best_solution, best_cost):
    final_route = []
    for i in best_solution:
        final_route.append(cities_mapping[i])
    final_route = [starting_point] + final_route + [starting_point]
    print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))

In [333]:
def perform_genetic_algorithm(no_of_iterations = 20, no_of_chromosomes = 100 ,mutation_factor=1, xover_factor = 0.3):
    best_solution = []
    # initialize the population and calculate the fitness criteria
    current_generation = initialPopCost(no_of_chromosomes)
    sorted_current_generation = current_generation.sort_values('Cost')
    best_cost = sorted_current_generation.iloc[0,1]
    for i in range(0, no_of_iterations) :
        start = datetime.now()
        
        mutation_factor = mutation_factor/float(i+1) # discounting
        
        print("mutation_factor", mutation_factor)
        next_generation = genetic_algorithm(sorted_current_generation,no_of_chromosomes  ,mutation_factor, xover_factor)
        print ("GA with and initial population size of {} took {} seconds".format(no_of_chromosomes,(datetime.now()-start).total_seconds()))
        sorted_next_generation = next_generation.sort_values('Cost')
        print ("Completed {} runs of GA.".format(i + 1))
        print ("-------------------------------------------------------------------------------------------" )
        print ("below are the best chromosomes:")
        print(sorted_next_generation.head(3))
        sorted_current_generation = sorted_next_generation
        if best_cost > sorted_next_generation.iloc[0,1] :
            best_cost = sorted_next_generation.iloc[0,1]
            best_solution = sorted_next_generation.iloc[0,0]
        final_route = []
        for i in best_solution:
            final_route.append(cities_mapping[i])
        final_route = [starting_point] + final_route + [starting_point]
        print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))
    print("Generated {}% of the {} solutions".format(np.round((next_generation.shape[0]/possible_solutions)*100,3),possible_solutions))
    get_final_rounte(best_solution, best_cost)
    
    return (best_solution, best_cost)

In [334]:
best_solution, best_cost = perform_genetic_algorithm(no_of_iterations = 3, no_of_chromosomes=10)

mutation_factor 1.0
Running crossover
Running Mutation
GA with and initial population size of 10 took 0.04098 seconds
Completed 1 runs of GA.
-------------------------------------------------------------------------------------------
below are the best chromosomes:
                             Route   Cost
2  [8, 1, 4, 7, 3, 2, 5, 10, 6, 9]  12936
7  [7, 1, 2, 3, 10, 6, 9, 5, 4, 8]  13771
9  [7, 1, 2, 3, 10, 6, 9, 5, 4, 8]  13771
Total distance travelled to cover the final route of 
 Athens => Gibraltar => Barcelona => Cherbourg => Geneva => Calais => Brussels => Cologne => Hook of Holland => Copenhagen => Hamburg => Athens is 12936
mutation_factor 0.5
Running crossover
Running Mutation
GA with and initial population size of 10 took 0.048975 seconds
Completed 2 runs of GA.
-------------------------------------------------------------------------------------------
below are the best chromosomes:
                             Route   Cost
8  [4, 7, 3, 2, 5, 9, 6, 10, 8, 1]  13451
0  [2, 3

In [335]:
!pip install --upgrade plotly

Requirement already up-to-date: plotly in c:\users\ise\anaconda3\lib\site-packages (4.3.0)


In [336]:
!pip install geopy

In [337]:
import plotly.graph_objects as go
import pandas as pd
from geopy.geocoders import Nominatim

In [338]:
geolocator = Nominatim(user_agent="GA",country_bias=['GB','DE','DK','GI','GR','ES','BE','FR','CH','NL'])
location = geolocator.geocode("Barcelona")
print(location.latitude,location.longitude)

C:\Users\ISE\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning:

`country_bias` argument of the Nominatim.__init__ is deprecated and will be removed in geopy 2.0. Use Nominatim.geocode(country_codes=['GB', 'DE', 'DK', 'GI', 'GR', 'ES', 'BE', 'FR', 'CH', 'NL']) instead.



41.3828939 2.1774322


In [339]:
list(data1.columns)

['Athens',
 'Barcelona',
 'Brussels',
 'Calais',
 'Cherbourg',
 'Cologne',
 'Copenhagen',
 'Geneva',
 'Gibraltar',
 'Hamburg',
 'Hook of Holland']

In [340]:
lat_long = []
for i in data1.columns:
    location = geolocator.geocode(i)
    lat_long.append([i,location.latitude,location.longitude])

In [341]:
lat_long = pd.DataFrame(lat_long,columns = ["city",'lat','long'])

In [342]:
lat_long

,city,lat,long
0,Athens,37.984149,23.727984
1,Barcelona,41.382894,2.177432
2,Brussels,50.846557,4.351697
3,Calais,50.948800,1.874680
4,Cherbourg,49.512720,-1.575544
5,Cologne,50.938361,6.959974
6,Copenhagen,55.686724,12.570072
7,Geneva,46.201756,6.146601
8,Gibraltar,36.140807,-5.354129
9,Hamburg,53.550341,10.000654


In [343]:
best_route_names=["Athens"]+[cities_mapping[i] for i in best_solution]+["Athens"]

In [344]:
#!pip install gmplot

In [345]:
import gmplot 

In [404]:
latitude_list = lat_long['lat']
longitude_list =lat_long['long'] 
  
gmap3 = gmplot.GoogleMapPlotter(max(lat_long['lat']), 
                                max(lat_long['long']),5) 
  

In [405]:
# scatter method of map object  
# scatter points on the google map 
gmap3.scatter( latitude_list, longitude_list, '# FF0000', 
                              size = 40, marker = False ) 

In [406]:
lat_long

,city,lat,long
0,Athens,37.984149,23.727984
1,Barcelona,41.382894,2.177432
2,Brussels,50.846557,4.351697
3,Calais,50.948800,1.874680
4,Cherbourg,49.512720,-1.575544
5,Cologne,50.938361,6.959974
6,Copenhagen,55.686724,12.570072
7,Geneva,46.201756,6.146601
8,Gibraltar,36.140807,-5.354129
9,Hamburg,53.550341,10.000654


In [407]:
best_route_names

['Athens',
 'Gibraltar',
 'Barcelona',
 'Cherbourg',
 'Geneva',
 'Calais',
 'Brussels',
 'Cologne',
 'Hook of Holland',
 'Copenhagen',
 'Hamburg',
 'Athens']

In [408]:
inverse_dict = {v: k for k, v in cities_mapping.items()}

In [409]:
inverse_dict

{'Athens': 0,
 'Barcelona': 1,
 'Brussels': 2,
 'Calais': 3,
 'Cherbourg': 4,
 'Cologne': 5,
 'Copenhagen': 6,
 'Geneva': 7,
 'Gibraltar': 8,
 'Hamburg': 9,
 'Hook of Holland': 10}

In [410]:
lat_list = []
long_list = []
for i in best_route_names:
    index = int(inverse_dict[i])
    lat_list.append(lat_long['lat'][index])
    long_list.append(lat_long['long'][index])
print(lat_list)
print("#####################################")
print(long_list)

[37.9841493, 36.140807, 41.3828939, 49.51271975, 46.2017559, 50.9488, 50.8465573, 50.938361, 51.9771615, 55.6867243, 53.550341, 37.9841493]
#####################################
[23.7279843, -5.3541295, 2.1774322, -1.57554410903806, 6.1466014, 1.87468, 4.351697, 6.959974, 4.1314526, 12.5700724, 10.000654, 23.7279843]


In [411]:

# Plot method Draw a line in 
# between given coordinates 
gmap3.plot(lat_list, long_list,  
           'red', edge_width = 2.5,) 

In [412]:
import datetime
file_name = datetime.datetime.now()

In [413]:
str(file_name.microsecond)

'634709'

In [414]:
gmap3.draw( str(file_name.microsecond)+"_map"+".html") 